In [0]:
SELECT
  to_date(CAST(a.DateInt as string), 'yyyyMMdd') as dateAsDate,
  --a.EPMBranch,
  b.branch_name as Entity,
  b.operating_unit_name as Country,
  CASE
    WHEN CostType = 'COS DIRECT' THEN 'DIRECT USD'
    WHEN CostType = 'COS INDIRECT' THEN 'INDIRECT USD'
  END as Account,
  format_number(ROUND(sum(AmountAnnual), 3), '$#,##0') as value,
  grouping_id(a.DateInt, a.CostType, b.operating_unit_name, b.branch_name) as grp_id,
  'Headcount_USD' as source
FROM
  `na-cdp-bu-hr-dev`.curated.usdamount as a
    LEFT JOIN (
      SELECT
        branch_no,
        branch_name,
        operating_unit_name
      FROM
        `na-da-dbr-shared-dev`.edw.dim_branch
      WHERE
        operating_unit_name IN ('CANADA', 'US')
    ) as b
      on b.branch_no = a.EPMBranch
WHERE
  DateInt >= 20231001
  and AmountAnnual <> 0
  and CostType IN ('COS DIRECT', 'COS INDIRECT')
GROUP BY
  GROUPING SETS (
    (a.DateInt, a.CostType, b.operating_unit_name, b.branch_name), (
      a.DateInt,
      a.CostType,
      b.operating_unit_name
    )
  );

SELECT
  EPMBranch,
  Breakdown2Id,
  Costtype,
  AmountAnnual
FROM
  `na-cdp-bu-hr-dev`.curated.usdamount
WHERE
  EPMBranch = 100552
  and dateint = 20241130;
-- /*  Research Branch Names and IDs */
--   SELECT
--         branch_no,
--         branch_name,
--         operating_unit_name
--       FROM
--         `na-da-dbr-shared-dev`.edw.dim_branch
--       WHERE
--         branch_name like 'NORTHEAST%'
--         --perating_unit_name IN ('CANADA', 'US')